# Machine Learnig Bootcamp
# HomeWork 8

# Dataset
In this homework, we'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from Kaggle and slightly rebuilt.<br>

You can download the target dataset for this homework from here:<br>

wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip<br>
unzip data.zip<br>
In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.<br>

We will use PyTorch for that.<br>

You can use Google Colab or your own computer for that.<br>

In [5]:

!unzip -q data.zip

# Data Preparation<br>
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.<br>


# Reproducibility<br>
Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same <br>results during the same experiment runs.<br>

Therefore, in this homework we suggest to set the random number seed generators by:<br>

import numpy as np<br>
import torch<br>

SEED = 42<br>
np.random.seed(SEED)<br>
torch.manual_seed(SEED)<br>

if torch.cuda.is_available():<br>
    torch.cuda.manual_seed(SEED)<br>
    torch.cuda.manual_seed_all(SEED)<br>

torch.backends.cudnn.deterministic = True<br>
torch.backends.cudnn.benchmark = False<br>
Also, use PyTorch of version 2.8.0 (that's the one in Colab).<br>

In [6]:
import numpy as np
import torch

SEED = 42

# Python, NumPy, PyTorch
np.random.seed(SEED)
torch.manual_seed(SEED)

# GPU-specific seeds
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# cuDNN reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [7]:
print(torch.__version__)

2.8.0+cpu


# Model<br>
For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.<br>

You need to develop the model with following structure:<br>

The shape for input should be (3, 200, 200) (channels first format in PyTorch)<br>
Next, create a convolutional layer (nn.Conv2d):<br>
Use 32 filters (output channels)<br>
Kernel size should be (3, 3) (that's the size of the filter)<br>
Use 'relu' as activation<br>
Reduce the size of the feature map with max pooling (nn.MaxPool2d)<br>
Set the pooling size to (2, 2)<br>
Turn the multi-dimensional result into vectors using flatten or view<br>
Next, add a nn.Linear layer with 64 neurons and 'relu' activation<br>
Finally, create the nn.Linear layer with 1 neuron - this will be the output<br>
The output layer should have an activation - use the appropriate activation for the binary classification case<br>
As optimizer use torch.optim.SGD with the following parameters:<br>

torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)<br>

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class HairCNN(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.conv = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3
        )

        
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )

        
        self.fc1 = nn.Linear(32 * 99 * 99, 64)  
        self.fc2 = nn.Linear(64, 1)             

    def forward(self, x):
        
        x = self.conv(x)        
        x = F.relu(x)
        x = self.pool(x)        

        x = x.view(x.size(0), -1)  

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = torch.sigmoid(x)     
        return x


In [10]:
model = HairCNN()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)

# Question 1 <br>
Which loss function you will use?<br>
<br>
nn.MSELoss()<br>
nn.BCEWithLogitsLoss()<br>
nn.CrossEntropyLoss()<br>
nn.CosineEmbeddingLoss()<br>

In [11]:
nn.BCEWithLogitsLoss()

BCEWithLogitsLoss()

# Question 2 <br>
What's the total number of parameters of the model? You can use torchsummary or count manually.<br>
<br>
In PyTorch, you can find the total number of parameters using:<br>

Option 1: Using torchsummary (install with: pip install torchsummary)<br>
from torchsummary import summary<br>
summary(model, input_size=(3, 200, 200))<br>
<br>
Option 2: Manual counting<br>
total_params = sum(p.numel() for p in model.parameters())<br>
print(f"Total parameters: {total_params}")<br>
896<br>
11214912<br>
15896912<br>
20073473<br>

In [12]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


# Generators and Training<br>
For the next two questions, use the following transformation for both train and test sets:<br>

train_transforms = transforms.Compose([<br>
    transforms.Resize((200, 200)),<br>
    transforms.ToTensor(),<br>
    transforms.Normalize(<br>
        mean=[0.485, 0.456, 0.406],<br>
        std=[0.229, 0.224, 0.225]<br>
    ) # ImageNet normalization<br>
])<br>
We don't need to do any additional pre-processing for the images.<br>
Use batch_size=20<br>
Use shuffle=True for both training, but False for test.<br>
Now fit the model.<br>
<br>
You can use this code:<br>
<br>
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}<br>

for epoch in range(num_epochs):<br>
    model.train()<br>
    running_loss = 0.0<br>
    correct_train = 0<br>
    total_train = 0<br>
    for images, labels in train_loader:<br>
        images, labels = images.to(device), labels.to(device)<br>
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)<br>
<br>
        optimizer.zero_grad()<br>
        outputs = model(images)<br>
        loss = criterion(outputs, labels)<br>
        loss.backward()<br>
        optimizer.step()<br>

        running_loss += loss.item() * images.size(0)<br>
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy<br>
        predicted = (torch.sigmoid(outputs) > 0.5).float()<br>
        total_train += labels.size(0)<br>
        correct_train += (predicted == labels).sum().item()<br>
<br>
    epoch_loss = running_loss / len(train_dataset)<br>
    epoch_acc = correct_train / total_train<br>
    history['loss'].append(epoch_loss)<br>
    history['acc'].append(epoch_acc)<br>

    model.eval()<br>
    val_running_loss = 0.0<br>
    correct_val = 0<br>
    total_val = 0<br>
    with torch.no_grad():<br>
        for images, labels in validation_loader:<br>
            images, labels = images.to(device), labels.to(device)<br>
            labels = labels.float().unsqueeze(1)<br>
            outputs = model(images)<br>
            loss = criterion(outputs, labels)<br>
<br>
            val_running_loss += loss.item() * images.size(0)<br>
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)<br>
            correct_val += (predicted == labels).sum().item()<br>

    val_epoch_loss = val_running_loss / len(validation_dataset)<br>
    val_epoch_acc = correct_val / total_val<br>
    history['val_loss'].append(val_epoch_loss)<br>
    history['val_acc'].append(val_epoch_acc)<br>

    print(f"Epoch {epoch+1}/{num_epochs}, "<br>
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "<br>
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"))<br>

In [14]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

NameError: name 'train_loader' is not defined

# Question 3 <br>
What is the median of training accuracy for all the epochs for this model?<br>
<br>
0.05<br>
0.12<br>
0.40<br>
0.84<br>

# Data Augmentation
For the next two questions, we'll generate more data using data augmentations.<br>
<br>
Add the following augmentations to your training data generator:<br>
<br>
transforms.RandomRotation(50),<br>
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),<br>
transforms.RandomHorizontalFlip(),<br>